# Распаковка данных от парсера

In [1]:
import json
import pandas as pd
import re

# Читаем файлик от парсера
with open('flats.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Helper functions for cleaning/extracting data
def clean_area(area_str):
    """Remove non-breaking spaces and ' м²', replace comma with dot,
       then convert to a number. If the result is an integer, cast to int."""
    if area_str:
        area_str = area_str.replace('\xa0', ' ').replace(" м²", "").strip()
        area_str = area_str.replace(",", ".")
        value = float(area_str)
        if value.is_integer():
            return int(value)
        return value
    return None

def parse_floor(floor_str):
    """Split floor string like '1 из 5' into (flat_floor, total_floors)."""
    parts = re.split(r'из', floor_str)
    if len(parts) == 2:
        flat_floor = int(parts[0].strip())
        total_floors = int(parts[1].strip())
        return flat_floor, total_floors
    return None, None

def extract_metro_time(metro_dict):
    """Return the minimum travel time from the metro dictionary, ignoring None values."""
    if metro_dict:
        valid_times = [t for t in metro_dict.values() if t is not None]
        if valid_times:
            return min(valid_times)
    return None

def extract_rooms(title):
    """Extract the number of rooms from the title.
       Return 'студия' if mentioned, otherwise the digit preceding '-комн.'."""
    if "студия" in title.lower():
        return "студия"
    match = re.search(r'(\d+)-\s*комн', title.lower())
    if match:
        return int(match.group(1))
    return None

def clean_ceiling_height(height_str):
    """Clean and convert ceiling height string like '3\xa0м' to a float."""
    if height_str:
        height_str = height_str.replace('\xa0', ' ').replace(" м", "").strip().replace(",", ".")
        return float(height_str)
    return None

def extract_int(value_str):
    """Extract the first integer found in the string."""
    if value_str:
        digits = re.findall(r'\d+', value_str)
        if digits:
            return int(digits[0])
    return None

# Process each record and build a list of rows for the DataFrame
rows = []
for d in data:
    row = {}
    # Top-level fields
    row["okrug"] = d.get("okrug")
    row["raion"] = d.get("raion")
    row["price"] = d.get("price")
    row["living_complex"] = d.get("living_complex")
    row["title"] = d.get("title")
    row["rooms"] = extract_rooms(d.get("title", ""))
    
    # Process metro: take the closest (minimum) time
    row["metro_time"] = extract_metro_time(d.get("metro", {}))
    
    # Process 'info' fields
    info = d.get("info", {})
    row["info_total_area"] = clean_area(info.get("Общая площадь"))
    row["info_living_area"] = clean_area(info.get("Жилая площадь"))
    row["info_kitchen_area"] = clean_area(info.get("Площадь кухни"))
    flat_floor, total_floors = parse_floor(info.get("Этаж", ""))
    row["info_floor"] = flat_floor
    row["info_total_floors"] = total_floors
    row["info_year_built"] = int(info.get("Год постройки").strip()) if info.get("Год постройки") else None
    
    # Process 'flat_details' fields:
    fd = d.get("flat_details", {})
    # Define keys to extract with mapping: JSON key -> (column name, processing function)
    fd_keys_to_extract = {
        "Общая площадь": ("fd_total_area", clean_area),
        "Жилая площадь": ("fd_living_area", clean_area),
        "Площадь кухни": ("fd_kitchen_area", clean_area),
        "Высота потолков": ("ceiling_height", clean_ceiling_height),
        "Санузел": ("bathroom", lambda x: x),
        "Вид из окон": ("view", lambda x: x),
        "Ремонт": ("repair", lambda x: x),
        "Год постройки": ("fd_year_built", lambda x: int(x.strip()) if x else None),
        "Строительная серия": ("building_series", lambda x: x),
        "Мусоропровод": ("waste_chute", lambda x: x),
        "Тип дома": ("house_type", lambda x: x),
        "Тип перекрытий": ("ceiling_type", lambda x: x),
        "Парковка": ("parking", lambda x: x),
        "Подъезды": ("entrances", extract_int),
        "Отопление": ("heating", lambda x: x),
        "Аварийность": ("emergency", lambda x: x),
        "Газоснабжение": ("gas_supply", lambda x: x),
        "Тип жилья": ("housing_type", lambda x: x),
        "Балкон/лоджия": ("balcony", extract_int),
        "Количество лифтов": ("num_lifts", extract_int)
    }
    

    # Process 'rent_details' fields (new fields)
    rent = d.get("rent_details", {})
    row["rent_price_per_month"] = rent.get("price_per_month")
    row["rent_jkh"] = rent.get("Оплата ЖКХ")
    row["rent_deposit"] = rent.get("Залог")
    row["rent_commissions"] = rent.get("Комиссии")
    row["rent_prepayment"] = rent.get("Предоплата")
    row["rent_term"] = rent.get("Срок аренды")
    row["rent_living_conditions"] = rent.get("Условия проживания")
    
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv('final_data_raw1.csv', index=False, encoding='utf-8')


# Импорт данных и библиотек

In [7]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('final_data_raw.csv')

# Предварительные данные

In [9]:
df.head(10)

,okrug,raion,price,living_complex,title,rooms,metro_time,info_total_area,info_living_area,info_kitchen_area,info_floor,info_total_floors,info_year_built,rent_price_per_month,rent_jkh,rent_deposit,rent_commissions,rent_prepayment,rent_term,rent_living_conditions
0,ЦАО,р-н Пресненский,470000,ЖК «Capital Towers (Капитал Тауэрс)»,"Сдается 2-комн. квартира, 73 м²",2,7.0,73.0,NaN,10.0,41,60,2023.0,470000,включена (счётчики включены),470 000 ₽,нет,1 месяц,от года,можно с детьми
1,ЦАО,р-н Пресненский,270000,ЖК «Пресня Сити»,"Сдается 3-комн. квартира, 68 м²",3,13.0,68.0,25.0,17.0,23,44,2018.0,270000,включена (без счётчиков),270 000 ₽,нет,1 месяц,от года,можно с детьми
2,СЗАО,р-н Митино,30000,NaN,"Сдается 1-комн. квартира, 40 м²",1,9.0,40.0,20.0,10.0,3,9,1995.0,30000,включена (счётчики включены),30 000 ₽,нет,1 месяц,от года,NaN
3,ЮАО,р-н Донской,75000,NaN,"Сдается 2-комн. квартира, 60 м²",2,13.0,60.0,NaN,NaN,3,8,1963.0,75000,включена (без счётчиков),75 000 ₽,нет,1 месяц,от года,NaN
4,САО,р-н Беговой,29000,ЖК «Царская площадь»,"Сдается апартаменты-студия, 12 м²",студия,7.0,12.0,6.0,2.0,11,18,2018.0,29000,включена (счётчики включены),9 000 ₽,нет,1 месяц,от года,NaN
5,ЦАО,р-н Басманный,550000,ЖК «TriBeCa APARTMENTS (Трайбека Апартментс)»,"Сдаются 3-комн. апартаменты, 150 м²",3,5.0,150.0,60.0,50.0,3,7,2015.0,550000,включена (без счётчиков),550 000 ₽,нет,1 месяц,от года,можно с детьми
6,ЗАО,р-н Фили-Давыдково,90000,NaN,"Сдается 1-комн. квартира, 43 м²",1,5.0,43.0,NaN,NaN,7,24,2005.0,90000,включена (без счётчиков),90 000 ₽,нет,1 месяц,от года,можно с детьми и животными
7,СЗАО,р-н Митино,59900,NaN,"Сдается апартаменты-студия, 29,2 м²",студия,9.0,29.2,25.0,2.0,19,24,2015.0,59900,включена (счётчики включены),50 000 ₽,нет,1 месяц,несколько месяцев,NaN
8,СВАО,р-н Останкинский,35000,NaN,"Сдается 2-комн. квартира, 36,7 м²",2,4.0,36.7,22.4,7.3,3,9,1962.0,35000,включена (счётчики включены),20 000 ₽,нет,1 месяц,несколько месяцев,NaN
9,ЮАО,р-н Даниловский,295000,ЖК «ЗИЛАРТ»,"Сдается 1-комн. квартира, 60,7 м²",1,11.0,60.7,40.0,15.0,12,14,2019.0,295000,включена (счётчики включены),295 000 ₽,нет,1 месяц,от года,можно с детьми и животными


# Описание и анализ признаков

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   okrug                   1405 non-null   object 
 1   raion                   1405 non-null   object 
 2   price                   1405 non-null   int64  
 3   living_complex          609 non-null    object 
 4   title                   1405 non-null   object 
 5   rooms                   1400 non-null   object 
 6   metro_time              1396 non-null   float64
 7   info_total_area         1405 non-null   float64
 8   info_living_area        1019 non-null   float64
 9   info_kitchen_area       1045 non-null   float64
 10  info_floor              1405 non-null   int64  
 11  info_total_floors       1405 non-null   int64  
 12  info_year_built         1016 non-null   float64
 13  rent_price_per_month    1405 non-null   int64  
 14  rent_jkh                1405 non-null   

## Okrug

Данный признак представляет собой наименование административного округа города Москвы, к которому относится объект недвижимости. Включает как центральные и периферийные округа, так и территории "Новой Москвы". Используется для пространственной локализации объекта и проведения географического анализа данных.

In [3]:
df['okrug'].nunique() # Количество округов

13

In [4]:
df['okrug'].unique()

array(['ЦАО', 'СЗАО', 'ЮАО', 'САО', 'ЗАО', 'СВАО', 'НАО (Новомосковский)',
       'ЮВАО', 'ЮЗАО', 'ВАО', 'ЗелАО', 'ТАО (Троицкий)',
       'р-н Новая Москва'], dtype=object)

## Raion

Данный признак отражает конкретный административный район в пределах округа Москвы или в составе территорий "Новой Москвы". Он предоставляет более детализированную географическую привязку объекта недвижимости и используется для углублённого пространственного анализа, в частности — для изучения локальных особенностей рынка аренды.










In [13]:
df['raion'].nunique() # Количество районов

178

In [14]:
df['raion'].unique()[0:10]

array(['р-н Пресненский', 'р-н Митино', 'р-н Донской', 'р-н Беговой',
       'р-н Басманный', 'р-н Фили-Давыдково', 'р-н Останкинский',
       'р-н Даниловский', 'р-н Таганский', 'Десеновское поселение'],
      dtype=object)

## Price

Признак Price представляет собой размер арендной платы за объект недвижимости, выраженный в рублях.

In [5]:
df = df.drop(columns=['rent_price_per_month'])

## living_complex

Признак living_complex отражает наименование жилого комплекса, в составе которого располагается объект аренды.

In [16]:
df['raion'].nunique() # Количество комплексов

178

In [17]:
df['living_complex'].unique()[0:10]

array(['ЖК «Capital Towers (Капитал Тауэрс)»', 'ЖК «Пресня Сити»', nan,
       'ЖК «Царская площадь»',
       'ЖК «TriBeCa APARTMENTS (Трайбека Апартментс)»', 'ЖК «ЗИЛАРТ»',
       'ЖК «на Калитниковской»', 'ЖК «Новые Ватутинки, мкр. Центральный»',
       'ЖК «RIVERDALE APARTMENTS (Ривердейл Апартаментс)»',
       'ЖК «Татьянин Парк»'], dtype=object)

In [6]:
df = df.drop(columns=['living_complex'])

## title

Краткое текстовое описание объекта недвижимости, включающее тип сделки (аренда или продажа), количество комнат, тип недвижимости (например, квартира), а также общую площадь жилого помещения в квадратных метрах.

In [7]:
df = df.drop(columns=['title'])

## rooms

Категориальный признак, указывающий число изолированных жилых помещений в квартире. Возможные значения включают числовые обозначения от 1 до 4, а также категорию «студия» (обозначающую открытую планировку без выделенных комнат).

In [20]:
df['rooms'].unique()

array(['2', '3', '1', 'студия', '4', nan], dtype=object)

## metro_time

metro_time — Минимальное время пешей доступности до ближайшей станции метро.
Числовой признак, измеряемый в минутах.

info_total_area — Общая площадь квартиры

Числовой признак, измеряемый в квадратных метрах. Отражает суммарную площадь всех помещений квартиры, включая жилые комнаты, кухню, коридоры, санузлы и другие вспомогательные пространства.

info_living_area — Жилая площадь квартиры

Числовой признак, измеряемый в квадратных метрах. Представляет собой площадь, непосредственно предназначенную для проживания (спальни, гостиные и другие жилые комнаты).

info_kitchen_area — Площадь кухни

In [8]:
df = df.drop(columns=['info_living_area'])
df = df.drop(columns=['info_kitchen_area'])

## info_floor info_total_floors

info_total_floors — Этажность здания. Общее количество этажей в здании, где расположена квартира.

info_floor - Конкретный этаж, на котором находится квартира в здании.

In [9]:
df = df.drop(columns=['info_total_floors'])

## info_year_built

Год ввода здания в эксплуатацию.

In [23]:
df['info_year_built'].unique()

array([2023., 2018., 1995., 1963., 2015., 2005., 1962., 2019., 2016.,
       2024., 2020.,   nan, 2010., 2022., 2006., 1966., 2021., 1953.,
       1968., 1961., 2008., 1964., 1976., 1971., 1978., 1969., 1970.,
       1973., 1965., 2003., 1960., 1959., 1957., 2013., 1967., 1983.,
       1986., 2012., 2017., 2009., 1991., 1972., 1974., 1979., 1956.,
       1975., 1937., 2011., 1994., 1977., 1987., 2014., 1982., 1980.,
       2007., 1992., 1984., 2004., 1998., 2000., 1999., 2002., 1988.,
       1989., 1929., 1940., 2001., 1981., 1927., 1954., 1990., 1934.,
       1931., 1985., 1997., 1928., 1958., 1996., 1993., 1955., 1900.,
       1890.])

## rent_jkh

Включены ли коммунальные платежи в стоимость аренды.

«включена (счётчики включены)» — стоимость аренды включает коммунальные услуги, оплата по счётчикам также включена.

«включена (без счётчиков)» — стоимость аренды включает фиксированную оплату коммунальных услуг без учета счётчиков.

In [24]:
df['rent_jkh'].unique()

array(['включена (счётчики включены)', 'включена (без счётчиков)'],
      dtype=object)

## rent_deposit

rent_deposit — это сумма залога за аренду квартиры, указанная в объявлении.
Как правило, залоговая сумма возвращается арендатору при окончании срока аренды при соблюдении условий договора (например, сохранность имущества, отсутствие задолженностей и т.п.)

In [10]:
df = df.drop(columns=['rent_deposit'])

## rent_commissions

In [26]:
df['rent_commissions'].unique()

array(['нет'], dtype=object)

In [11]:
df = df.drop(columns=['rent_commissions'])

## rent_prepayment

rent_prepayment — это срок предоплаты за аренду квартиры, выраженный в количестве месяцев.

In [28]:
df['rent_prepayment'].unique()

array(['1 месяц', '2 месяца', '3 месяца'], dtype=object)

## rent_term

In [29]:
df['rent_term'].unique()

array(['от года', 'несколько месяцев'], dtype=object)

rent_term — это срок аренды квартиры, указанный в объявлении.
Значение отражает, на какой период арендодатель планирует сдавать жильё.

## rent_living_conditions

rent_living_conditions — это условия проживания, связанные с возможностью содержания детей и/или животных в арендуемой квартире.

In [12]:
df['rent_living_conditions'] = df['rent_living_conditions'].fillna('ни с детьми, ни с животными')

In [31]:
df['rent_living_conditions'].unique()

array(['можно с детьми', 'ни с детьми, ни с животными',
       'можно с детьми и животными', 'можно с животными'], dtype=object)

In [32]:
df

,okrug,raion,price,rooms,metro_time,info_total_area,info_floor,info_year_built,rent_jkh,rent_prepayment,rent_term,rent_living_conditions
0,ЦАО,р-н Пресненский,470000,2,7.0,73.0,41,2023.0,включена (счётчики включены),1 месяц,от года,можно с детьми
1,ЦАО,р-н Пресненский,270000,3,13.0,68.0,23,2018.0,включена (без счётчиков),1 месяц,от года,можно с детьми
2,СЗАО,р-н Митино,30000,1,9.0,40.0,3,1995.0,включена (счётчики включены),1 месяц,от года,"ни с детьми, ни с животными"
3,ЮАО,р-н Донской,75000,2,13.0,60.0,3,1963.0,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными"
4,САО,р-н Беговой,29000,студия,7.0,12.0,11,2018.0,включена (счётчики включены),1 месяц,от года,"ни с детьми, ни с животными"
...,...,...,...,...,...,...,...,...,...,...,...,...
1400,ЮВАО,р-н Южнопортовый,65000,студия,5.0,27.0,4,2021.0,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными"
1401,САО,р-н Дмитровский,65000,1,24.0,35.7,21,2022.0,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными"
1402,СЗАО,р-н Северное Тушино,65000,1,6.0,40.0,14,2006.0,включена (без счётчиков),1 месяц,от года,можно с детьми
1403,ЗАО,р-н Очаково-Матвеевское,64000,студия,7.0,22.0,11,2024.0,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными"


# One-Hot-Encoding, пропуски, дубликаты

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   okrug                   1405 non-null   object 
 1   raion                   1405 non-null   object 
 2   price                   1405 non-null   int64  
 3   rooms                   1400 non-null   object 
 4   metro_time              1396 non-null   float64
 5   info_total_area         1405 non-null   float64
 6   info_floor              1405 non-null   int64  
 7   info_year_built         1016 non-null   float64
 8   rent_jkh                1405 non-null   object 
 9   rent_prepayment         1405 non-null   object 
 10  rent_term               1405 non-null   object 
 11  rent_living_conditions  1405 non-null   object 
dtypes: float64(3), int64(2), object(7)
memory usage: 131.8+ KB


In [13]:
def assign_year_interval(year):
    try:
        year = int(year)
        if 1890 <= year <= 1917:
            return '1890-1917'
        elif 1917 < year <= 1945:
            return '1917-1945'
        elif 1945 < year <= 1970:
            return '1945-1970'
        elif 1970 < year <= 1990:
            return '1970-1990'
        elif 1991 <= year <= 2025:
            return '1991-2025'
        else:
            return 'не попал'
    except:
        return 'не попал'

df['year_built_interval'] = df['info_year_built'].apply(assign_year_interval)

In [14]:
df = df.drop(columns=['info_year_built'])

In [15]:
df = df.dropna()

df = df.drop_duplicates()

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1390 entries, 0 to 1404
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   okrug                   1390 non-null   object 
 1   raion                   1390 non-null   object 
 2   price                   1390 non-null   int64  
 3   rooms                   1390 non-null   object 
 4   metro_time              1390 non-null   float64
 5   info_total_area         1390 non-null   float64
 6   info_floor              1390 non-null   int64  
 7   rent_jkh                1390 non-null   object 
 8   rent_prepayment         1390 non-null   object 
 9   rent_term               1390 non-null   object 
 10  rent_living_conditions  1390 non-null   object 
 11  year_built_interval     1390 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 141.2+ KB
None


In [16]:
columns_to_encode = [
    'okrug',
    'raion',
    'rooms',
    'rent_jkh',
    'info_floor',
    'rent_prepayment',
    'rent_term',
    'rent_living_conditions',
    'year_built_interval'
]

df_encoded = pd.get_dummies(df, columns=columns_to_encode, prefix=columns_to_encode).astype(int)

In [38]:
df

,okrug,raion,price,rooms,metro_time,info_total_area,info_floor,rent_jkh,rent_prepayment,rent_term,rent_living_conditions,year_built_interval
0,ЦАО,р-н Пресненский,470000,2,7.0,73.0,41,включена (счётчики включены),1 месяц,от года,можно с детьми,1991-2025
1,ЦАО,р-н Пресненский,270000,3,13.0,68.0,23,включена (без счётчиков),1 месяц,от года,можно с детьми,1991-2025
2,СЗАО,р-н Митино,30000,1,9.0,40.0,3,включена (счётчики включены),1 месяц,от года,"ни с детьми, ни с животными",1991-2025
3,ЮАО,р-н Донской,75000,2,13.0,60.0,3,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными",1945-1970
4,САО,р-н Беговой,29000,студия,7.0,12.0,11,включена (счётчики включены),1 месяц,от года,"ни с детьми, ни с животными",1991-2025
...,...,...,...,...,...,...,...,...,...,...,...,...
1400,ЮВАО,р-н Южнопортовый,65000,студия,5.0,27.0,4,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными",1991-2025
1401,САО,р-н Дмитровский,65000,1,24.0,35.7,21,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными",1991-2025
1402,СЗАО,р-н Северное Тушино,65000,1,6.0,40.0,14,включена (без счётчиков),1 месяц,от года,можно с детьми,1991-2025
1403,ЗАО,р-н Очаково-Матвеевское,64000,студия,7.0,22.0,11,включена (без счётчиков),1 месяц,от года,"ни с детьми, ни с животными",1991-2025


In [39]:
df_encoded

,price,metro_time,info_total_area,okrug_ВАО,okrug_ЗАО,okrug_ЗелАО,okrug_НАО (Новомосковский),okrug_САО,okrug_СВАО,okrug_СЗАО,...,rent_living_conditions_можно с детьми,rent_living_conditions_можно с детьми и животными,rent_living_conditions_можно с животными,"rent_living_conditions_ни с детьми, ни с животными",year_built_interval_1890-1917,year_built_interval_1917-1945,year_built_interval_1945-1970,year_built_interval_1970-1990,year_built_interval_1991-2025,year_built_interval_не попал
0,470000,7,73,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,270000,13,68,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,30000,9,40,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,75000,13,60,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,29000,7,12,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,65000,5,27,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1401,65000,24,35,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1402,65000,6,40,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1403,64000,7,22,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [17]:
print(df_encoded.isnull().any().any())
#Пропуски
print(df_encoded.duplicated().sum().sum())
#Дубликаты

False
0


In [44]:
print(df_encoded.dtypes)

price                            int64
metro_time                       int64
info_total_area                  int64
okrug_ВАО                        int64
okrug_ЗАО                        int64
                                 ...  
year_built_interval_1917-1945    int64
year_built_interval_1945-1970    int64
year_built_interval_1970-1990    int64
year_built_interval_1991-2025    int64
year_built_interval_не попал     int64
Length: 253, dtype: object


In [ ]:
df_encoded.to_csv('final_encoded.csv', index=False)
print("Файл сохранён как final_encoded.csv")

Файл сохранён как final_encoded.csv
